# Métodos de optimización no lineal sin restricciones
----------------

## Autores
* Quesada, Francisco
* Serpe, Octavio
* Arca, Gonzalo

-----------------

## Librerías utilizadas
* SciPy
* NumPy
-----------------


In [80]:
import math
from scipy.optimize import minimize
import numpy as np
import time

In [2]:
MIN_REAL = -10
MAX_REAL = 10

NUMBER_OF_PARAMETERS = 11

DATASET_INPUT = [(4.4793, -4.0765, -4.0765), (-4.1793, -
                                              4.9218, 1.7664), (-3.9429, -0.7689, 4.8830)]

DATASET_OUTPUT = [0, 1, 1]

In [7]:
# W 3 element vector
# w 2x3 matrix as 6 element vector
# w_0 2 element vector
# reactive_values 3 element vector
def f(W, w, w_0, reactive_values):
    inner_g_input = 0
    outer_g_input = 0

    for j in range(2):
        for k in range(3):
            inner_g_input += w[k + (j * 3)] * reactive_values[k] - w_0[j]
        outer_g_input += W[j] * g(inner_g_input)
    return g(outer_g_input - W[0])


# dataset_input 3 element vector
# dataset_output 3 element vector
def error(x, dataset_input, dataset_output):
    #dataset_input, dataset_output = dataset_tuple
    error = 0

    W = x[0:3]
    w = x[3:9]
    w_0 = x[9:11]

    for i in range(len(dataset_input)):
        error += math.pow(dataset_output[i] -
                          f(W, w, w_0, dataset_input[i]), 2)

    return error


def g(x):
    # If over the limit, return the limit value
    try:
        exp_value = math.exp(x)
    except OverflowError:
        return 1

    return exp_value / (1 + exp_value)

## Método 1: Gradiente descendiente (método quasi-Newton BFGS)

In [132]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html

x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
start_time = time.perf_counter()
optimizeResult = minimize(error, x0, method='BFGS', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-10, options={"disp":True})
end_time = time.perf_counter()
print(f'Argumentos óptimos:\nW = \t{optimizeResult.x[0:3]}\nw = \t{optimizeResult.x[3:6]}\n\t{optimizeResult.x[6:9]}\nw_0 = \t{optimizeResult.x[9:11]}\n')
print(f'error: {error(optimizeResult.x, DATASET_INPUT, DATASET_OUTPUT)}')
print("Tiempo de ejecucion:", end_time - start_time , "segundos")

Optimization terminated successfully.
         Current function value: 0.250000
         Iterations: 9
         Function evaluations: 240
         Gradient evaluations: 20
Argumentos óptimos:
W = 	[-12.34575269   1.86210741   7.2390848 ]
w = 	[ 3.331667    9.89523028 -5.44874473]
	[-7.19092443  7.38172134  6.92443013]
w_0 = 	[-9.09880461  5.41392854]

error: 0.25000000012544193
Tiempo de ejecucion: 0.008009666998987086 segundos


## Método 2: Gradientes conjugados

In [102]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-cg.html

x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
start_time = time.perf_counter()
optimizeResult = minimize(error, x0, method='CG', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-10, options={"disp":True})
end_time = time.perf_counter()
print(f'Argumentos óptimos:\nW = \t{optimizeResult.x[0:3]}\nw = \t{optimizeResult.x[3:6]}\n\t{optimizeResult.x[6:9]}\nw_0 = \t{optimizeResult.x[9:11]}\n')
print(f'error: {error(optimizeResult.x, DATASET_INPUT, DATASET_OUTPUT)}')
print("Tiempo de ejecucion:", end_time - start_time , "segundos")

Optimization terminated successfully.
         Current function value: 0.500000
         Iterations: 28
         Function evaluations: 1428
         Gradient evaluations: 119
Argumentos óptimos:
W = 	[5.87088232e-09 1.08457525e+01 9.97618221e+00]
w = 	[9.3526285  9.92319251 8.4732359 ]
	[-7.20631308  7.31355991  9.5537783 ]
w_0 = 	[ 7.50044359 -4.74050932]

0.5000000003797364
Tiempo de ejecucion: 0.03835148200050753 segundos


## Método 3: ADAM

### Implementación del método

In [ ]:
# insert implementación de método ADAM

### Uso del método

In [ ]:
# insert uso del método ADAM